In [22]:
import numpy as np
import hail as hl
from hail import methods
import scipy as sp
import pandas as pd
from math import sqrt, pi

In [23]:
def fastSVD(A, m, n, k, q):
    
    # STAGE A
    
    omega = np.random.normal(0, 100, (n, 2*k))
    adjointA = A.getH()
    AA = A @ adjointA
    AOmega = A @ omega

    Y = AOmega
    for i in range(0, q):
        Y = AA @ Y

    (Q, R) = np.linalg.qr(Y)

    # STAGE B

    adjointQ = Q.getH()
    B = adjointQ @ A
    (tildeU, S, V) = np.linalg.svd(B)
    U = Q @ tildeU
    
    return U, S, V, Q, adjointQ

In [24]:
# Error bound formulas
# pass in singular values, m, n, k, p

# ||A - QQ*A|| < [1 + 9 sqrt(k + p) sqrt(min(m,n))] * k + 1st largest singular value of A
def errorBound1(S, m, n, k, p):
    singularVal = S[k]
    return (1 + 9 * sqrt(k + p) * sqrt(min(m,n))) * singularVal

# A Posteriori Error Estimation
def aPosterioriError(S, m, n, k, Q, adjointQ, A):
    r = 100
    diff = A - Q @ adjointQ @ A
    max = 0
    for i in range(0,r):
        norm = np.linalg.norm(diff @ np.random.normal(0, 1, (n, 1)), 2)
        if max < norm:
            max = norm
    return max

# Blanczos paper error bound 4.3
def blanczosError(U, S, V, m, n, k, q, A, k1th_sing_val):
    norm_diff = np.linalg.norm(A - U @ S @ V.transpose())
    bound = 100 * l * (((m-k)/l) ** (1/(4*q + 2))) * k1th_sing_val
    return norm_diff <= bound

In [25]:
# Run on random data

def makeRandomData():
    m = np.random.randint(50, 300)
    n = np.random.randint(50, 300)
    A = np.asmatrix(np.random.rand(m, n))
    k = int((m + n)/4) # this is based on nothing, just picking some k
    q = 2
    return (A, m, n, k, q)

A, m, n, k, q = makeRandomData()
U, S, V, Q, adjointQ = fastSVD(A, m, n, k, q)

# Look at differences:

(u, s, v) = np.linalg.svd(A)
simple_diff = s - S
#print("subtracted difference between singular values: ", simple_diff)
print("maximum singular value difference: ", max(simple_diff))

diff = A - Q @ adjointQ @ A
norm_diff = np.linalg.norm(diff, 2)
print("L2 norm error of ||A - QQ*A||: ", norm_diff)

maximum singular value difference:  8.881784197001252e-15
L2 norm error of ||A - QQ*A||:  1.6191411675362965e-14


In [26]:
# Define Hardy-Weinberg population normalization function (from John)

def hwe_normalize(call_expr):
    mt = call_expr._indices.source
    mt = mt.select_entries(__gt=call_expr.n_alt_alleles())
    mt = mt.annotate_rows(__AC=hl.agg.sum(mt.__gt),
                          __n_called=hl.agg.count_where(hl.is_defined(mt.__gt)))
    mt = mt.filter_rows((mt.__AC > 0) & (mt.__AC < 2 * mt.__n_called))

    n_variants = mt.count_rows()
    if n_variants == 0:
        raise FatalError("hwe_normalized: found 0 variants after filtering out monomorphic sites.")

    mt = mt.annotate_rows(__mean_gt=mt.__AC / mt.__n_called)
    mt = mt.annotate_rows(
        __hwe_scaled_std_dev=hl.sqrt(mt.__mean_gt * (2 - mt.__mean_gt) * n_variants / 2))
    mt = mt.unfilter_entries()

    normalized_gt = hl.or_else((mt.__gt - mt.__mean_gt) / mt.__hwe_scaled_std_dev, 0.0)
    return normalized_gt

In [27]:
# Import/generate genetic data in Hardy-Weinberg equilibrium
# to run algorithm on more-realistic data
mt = hl.balding_nichols_model(3, 1000, 10000)
norm_gt = hwe_normalize(mt.GT)
np_matrix = hl.linalg.BlockMatrix.from_entry_expr(norm_gt).to_numpy()

2020-07-13 11:38:31 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 1000 samples, and 10000 variants...
2020-07-13 11:38:33 Hail: INFO: Coerced sorted dataset
2020-07-13 11:38:36 Hail: INFO: Wrote all 3 blocks of 10000 x 1000 matrix with block size 4096.


In [28]:
# Run fastPCA on genetic data

A = np.asmatrix(np_matrix)
m, n = np_matrix.shape
k = int((m + n)/4)
q = 2

U, S, V, Q, adjointQ = fastSVD(A, m, n, k, q)

# Look at differences:

(u, s, v) = np.linalg.svd(A)
simple_diff = s - S
# print("subtracted difference between singular values: ", simple_diff)
print("maximum singular value difference: ", max(simple_diff))

diff = A - Q @ adjointQ @ A
norm_diff = np.linalg.norm(diff, 2)

# this method of error calculation not actually applicable?
# print("L2 norm error of ||A - QQ*A||: ", norm_diff)
# print(norm_diff < errorBound1(S, m, n, k, q))

print(norm_diff < aPosterioriError(S, m, n, k, Q, adjointQ, A))

maximum singular value difference:  4.187487943684641e-07
True


In [29]:
# Basic algorithm from Banczos paper
# HAS DIMENSIONALITY ISSUES, will fail

mt2 = hl.balding_nichols_model(3, 1000, 1000)
norm_gt2 = hwe_normalize(mt2.GT)
np_matrix2 = hl.linalg.BlockMatrix.from_entry_expr(norm_gt2).to_numpy()


# again we use:
A = np.asmatrix(np_matrix2)
m, n = np_matrix2.shape
k = int((m + n)/16)
q = 2
i = 5

print("m, n: ", m, n)
print("k: ", k)

assert 2*k < m <= n

# we choose an integer l > k such that l ≤ m − k
l = int(m/2)
print("l: ", l)
assert l > k and l <= m - k

G = np.random.normal(0, 1, (l, m))
AAt = A @ A.transpose()
AAti = AAt @ AAt
for j in range(0,i):
    AAti = AAti @ AAt
R = G @ AAti @ A
assert R.shape == (l, n)

# having dimension issues here because it does not use
# the method of creating Q from the paper
(Q, S) = np.linalg.qr(R.transpose())
print("Q", Q.shape, (l, k))
print("S", S.shape, (k, n))

(Ru, Rs, Rv) = np.linalg.svd(R)
print(Ru.shape, (n, l))
print(Rs.shape, l)
print(Rv.shape, (l, l))
s_sorted = np.sort(Rs)

# print(s_sorted)
# print(reverse(s))
# assert (s == s_sorted).all()

k_largest = s[k]
assert np.linalg.norm(Q @ S - R.transpose(), 2) < k_largest

T = A @ Q
print("T", T.shape, (m, k))
assert T.shape == (m, k)
(Tu, Ts, Tw) = np.linalg.svd(T)
V = Q @ Tw



2020-07-13 11:42:37 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 1000 samples, and 1000 variants...
2020-07-13 11:42:41 Hail: INFO: Coerced sorted dataset
2020-07-13 11:42:42 Hail: INFO: Wrote all 1 blocks of 1000 x 1000 matrix with block size 4096.


m, n:  1000 1000
k:  125
l:  500
Q (1000, 500) (500, 125)
S (500, 500) (125, 1000)
(500, 500) (1000, 500)
(500,) 500
(1000, 1000) (500, 500)
T (1000, 500) (1000, 125)


AssertionError: 

In [31]:
# Banczos algorithm

def blanczosSVD(A, m, n, k, l, q):

    assert l > k
    assert (q+1)*l <= (m - k)
    assert m <= n

    G = np.random.normal(0, 1, (l, m))
    R = G @ A
    #AtA = A.transpose() @ A
    listR = [R]
    for i in range(0, q):
        Ri = (listR[i] @ A.transpose()) @ A
        listR.append(Ri)
        R = np.concatenate((R, Ri), axis=0)

    (Q, S) = np.linalg.qr(R.transpose())
    assert(Q.shape == (n, (q+1)*l))
    assert(S.shape == ((q+1)*l, (q+1)*l))

    T = A @ Q
    assert(T.shape == (m, (q+1)*l))
    
    (Tu, Ts, Tw) = np.linalg.svd(T, full_matrices=False)
    assert(Tu.shape, (m, (q+1)*l))
    assert(Ts.shape == ((q+1)*l,))
    assert(Tw.shape == ((q+1)*l, (q+1)*l))
    
    
    V = Q @ Tw
    
    bound = blanczosError(Tu, np.diag(Ts), V, m, n, k, q, A, Ts[k])
    print("Satisfies Blanczos error bound equation 4.3: ", bound)

    return (Tu, Ts, V)


mt3 = hl.balding_nichols_model(3, 10000, 1000)
norm_gt3 = hwe_normalize(mt3.GT)
np_matrix3 = hl.linalg.BlockMatrix.from_entry_expr(norm_gt3).to_numpy()

A = np.asmatrix(np_matrix3)
(m, n) = A.shape
k = 50
l = k + 2
q = 8

(blanczosU, blanczosS, blanczosV) = blanczosSVD(A, m, n, k, l, q)

2020-07-13 11:46:13 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 10000 samples, and 1000 variants...
2020-07-13 11:46:17 Hail: INFO: Coerced sorted dataset
2020-07-13 11:46:20 Hail: INFO: Wrote all 3 blocks of 1000 x 10000 matrix with block size 4096.


Satisfies Blanczos error bound equation 4.3:  True


In [79]:
# Manual blocked implementation
# Uses direction of multiplication matching Patrick's notes instead of the paper

mt4 = hl.balding_nichols_model(3, 1000, 10000)
norm_gt4 = hwe_normalize(mt4.GT)
np_matrix4 = hl.linalg.BlockMatrix.from_entry_expr(norm_gt4).to_numpy()

A = np.asmatrix(np_matrix4)
(m, n) = A.shape
k = 50
l = k + 2
q = 0

def makeBlockedA(A):
    # doesn't make use of block_size yet...
    return [A[i,:] for i in range(0,m)]

# if C blocked by columns
def matmulColBlocked(C, D):
    prod = C[0] @ D
    for i in range(1, len(C)):
        prod = np.concatenate((prod, A[i] @ B), axis=0) 

def matmulRightRowBlocked(A, B):
    prod = A[0] @ B
    for i in range(1, m):
        prod = np.concatenate((prod, A[i] @ B), axis=0) 
    return prod

def matmulLeftRowBlocked(A, B):
    prod = B @ A[0]
    for i in range(0, m):
        prod = np.concatenate((prod, A[i] @ B), axis=0) 
    return prod


def blockedBlanczosSVD(A, blockedA, m, n, k, l, q):

    assert l > k
    assert (q+1)*l <= (m - k)
    assert n <= m

    G = np.random.normal(0, 1, (n,l))
    
    # if q = 0 this is all we need
    H = blockedA[0] @ G
    for i in range(1,m):
        H = np.concatenate((H, blockedA[i] @ G), axis=0) 
        
    assert(H.shape == (m, (q+1)*l))
    
    listH = [H]
    for i in range(0, q):
        Hi = A @ (A.transpose() @ listH[i] )
        listH.append(Hi)
        H = np.concatenate((H, Hi), axis=0)

    (Q, R) = np.linalg.qr(H)
    assert(Q.shape == (m, (q+1)*l))
    assert(R.shape == ((q+1)*l, (q+1)*l))

    print("Q", Q.shape)
    print("A", A.shape)
    
    T = Q.transpose() @ A
    assert(T.shape == ((q+1)*l), m)
    
    (Tu, Ts, Tw) = np.linalg.svd(T, full_matrices=False)
    assert(Tu.shape, (m, (q+1)*l))
    assert(Ts.shape == ((q+1)*l,))
    assert(Tw.shape == ((q+1)*l, (q+1)*l))
    
    
    V = Q @ Tw
    
    bound = blanczosError(Tu, np.diag(Ts), V, m, n, k, q, A, Ts[k])
    print("Satisfies Blanczos error bound equation 4.3: ", bound)

    return (Tu, Ts, V)


2020-07-15 11:30:39 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 1000 samples, and 10000 variants...
2020-07-15 11:30:44 Hail: INFO: Coerced sorted dataset
2020-07-15 11:30:49 Hail: INFO: Wrote all 3 blocks of 10000 x 1000 matrix with block size 4096.


In [77]:
blockedBlanczosSVD(makeBlockedA(A), m, n, k, l, q)

Q (10000, 52)


AttributeError: 'list' object has no attribute 'shape'